In [1]:
# default_exp frappe.suite

# Frappe suite

> Frappe is the package which has definItions for the test suite `deescribe` and tets function `It`. 

In [2]:
#hide
from nbdev.showdoc import *
from nbdev.export import notebook2script
from fastcore.imports import *
from fastcore.foundation import *
from fastcore.basics import *
from fastcore.meta import *
from fastcore.test import *

In [3]:
#export
from contextlib import ContextDecorator

In [4]:
_events = L.split('after_enter before_exit')

In [5]:
mk_class('event', **_events.map_dict(),
         doc="All possible events as attributes to get tab-completion and typo-proofing")

In [6]:
#export
@funcs_kwargs(as_method=True)
class Callback(Stateful,GetAttr):
    "Basic class handling tweaks of the runner"
    _methods = _events

    def __init__(self, **kwargs): assert not kwargs, f'Passed unknown events: {kwargs}'
    def __repr__(self): return type(self).__name__

    def __call__(self, event_name):
        "Call `self.{event_name}` if it's defined"
        res = getattr(self, event_name, noop)()
        return res

    def __setattr__(self, name, value):
        super().__setattr__(name, value)

    @property
    def name(self):
        "Name of the `Callback`, camel-cased and with '*Callback*' removed"
        return class2attr(self, 'Callback')

The `Callback` can be used to define callbacks to intorduce new tweaks into the test runner. Currently there are two life cycle events of the test runner.

* `after_enter` - called after entering into Describe or It.
* `before_exit` - called before exiting the Describe or It.

> The `Callback` class is hugely inspired by the beautiful FastAI library. Infact the code for the callback is the direct copy of the Callback class in the FastAI class. Some of the tests are also from the FastAI library.

A callback can be created by subclassing the `Callback` class like this👇

In [7]:
class _T(Callback):
    def call_me(self): return "maybe"
test_eq(_T()("call_me"), "maybe")

You can also create a function and then assign it to an event in the Callback class. Doing this would give you a callback for that event. 

👀 at the test below for an example about how to assign a function as a callback event.

In [8]:
def cb(self): return "maybe"
_t = Callback(after_enter=cb)
test_eq(_t(event.after_enter), "maybe")

In [ ]:
class CallbackHandler():
    "Basic handler to assign the callback events to their respective flows"
    def __init__(self,cbs=None): self.cbs = cbs if cbs else []

    def after_enter(self):
        res = False
        for cb in self.cbs: res = res and cb.after_enter()
        return res

    def before_exit(self):
        res = False
        for cb in self.cbs: res = res and cb.before_exit()
        return res

In [9]:
def sum():
    return 1+1

In [10]:
#export
class Describe(ContextDecorator):
    "contextmanager to describe a test suite"
    def __init__(self, desc): store_attr('desc', self)

    def __enter__(self):
        "start the execution of the suite and do setup"
        print(self.desc)
        return self
    #TODO: create callbacks to check type, value,traceback and format report and termnial display
    def __exit__(self, type, value, traceback):
        "cleanup and exit the suite"
        print(f'{self.desc} completed')
        #print(type, value, traceback, sep="\n")
        return True

In [11]:
show_doc(Describe.__enter__)

<h4 id="Describe.__enter__" class="doc_header"><code>Describe.__enter__</code><a href="__main__.py#L6" class="source_link" style="float:right">[source]</a></h4>

> <code>Describe.__enter__</code>()

start the execution of the suite and do setup

In [12]:
show_doc(Describe.__exit__)

<h4 id="Describe.__exit__" class="doc_header"><code>Describe.__exit__</code><a href="__main__.py#L11" class="source_link" style="float:right">[source]</a></h4>

> <code>Describe.__exit__</code>(**`type`**, **`value`**, **`traceback`**)

cleanup and exit the suite

In [13]:
@Describe("fake failing test")
def test():
    return test_eq(sum(),1)
    
test()

fake failing test
fake failing test completed


This won't work👇

In [14]:
with Describe("fake negative test") as d:
    d.test_ne(sum(), 1)

fake negative test
fake negative test completed


This works👇

In [15]:
with Describe("positive fake test suite"):
    test_eq(sum(),2)

positive fake test suite
positive fake test suite completed


In [16]:
with Describe("first describe"):
    print('I am inside first describe')
    with Describe("second describe") as d:
        test_eq(sum(),2)

first describe
I am inside first describe
second describe
second describe completed
first describe completed


In [17]:
@Describe("fake positive test")
def postest():
    test_eq(sum(), 2)

postest()

fake positive test
fake positive test completed


In [18]:
@Describe("first describe")
@Describe("second describe")
def postest():
    test_eq(sum(), 2)

postest()

first describe
second describe
second describe completed
first describe completed


In [19]:
#export
class It(ContextDecorator):
    "contextmanager to describe a test function"
    def __init__(self, desc): store_attr('desc', self)

    def __enter__(self):
        "start the execution of the test"
        print(self.desc)
        return self
    #TODO: create callbacks to check type, value,traceback and format report and termnial display
    def __exit__(self, type, value, traceback):
        "cleanup and exit the test"
        print(f'{self.desc} completed')
        #print(type, value, traceback, sep="\n")
        return True

In [20]:
with Describe("fake test suite"):
    with It("should have some fake result"):
        test_eq(sum(),2)

fake test suite
should have some fake result
should have some fake result completed
fake test suite completed


In [21]:
with Describe("Fake posItive test suite"):
    with It("should have positive test result"):
        test_eq(sum(), 2)

Fake posItive test suite
should have positive test result
should have positive test result completed
Fake posItive test suite completed


In [22]:
with Describe("Fake posItive test suite"):
    with It("should have posItive test result"):
        test_eq(sum(), 2)

    with It("should have negative test result"):
        test_ne(sum(),1)

Fake posItive test suite
should have posItive test result
should have posItive test result completed
should have negative test result
should have negative test result completed
Fake posItive test suite completed


In [23]:
@Describe("Fake positive test suite")
@It("should have positive test result")
def postest():
    return test_eq(sum(), 2)

postest()

Fake positive test suite
should have positive test result
should have positive test result completed
Fake positive test suite completed


In [24]:
@Describe("Fake negative test suite")
@It("should have negative test result")
def negtest():
    return test_ne(sum(), 1)

negtest()

Fake negative test suite
should have negative test result
should have negative test result completed
Fake negative test suite completed


In [25]:
notebook2script()

Converted 00_frappe.suite.ipynb.
